### Importación de todas la librerias utilizadas.

In [1]:
# Libreria para el manejo de excepciones en Python
import sys

# Libreria para el manejo de datos
import pandas as pd

# Librería propietaria utilitaria
import BayesLibUtils as blU

# Librería que contiene los modelos Bayesianos 
import ModelosLib as modelos

# Libreria para discretizar variables
from sklearn.preprocessing import KBinsDiscretizer

# Flag que permite visualizar o no las salidas de cada operación
outInfo = False

# Permite elegir con que modelo implementado realizar la Red Bayesiana
# "Python" == 0
# "R" == 1
flagModelo = 1

# Este flag solo está diseñado para operar cuando el valor "flagModelo == 1"
# True = permite usar el modelo con todas las variables discretas
# False = permite usar el modelo con variables mixtas, es decir, discretas y continuas
discreta = False 

# Este flag solo está diseñado para operar cuando el valor "flagModelo == 1"
# Permite indicar al aprendizaje de Estructura de "R" que algoritmo de puntuación utilizar
# Algoritmos de puntuación disponibles:
#   1. score = "aic"       -> Solo para variables discretas, es decir, cuando flag "discreta = True"
#   2. score = "bic"       -> Solo para variables discretas, es decir, cuando flag "discreta = True"
#   3. score = "loglik"    -> Solo para variables discretas, es decir, cuando flag "discreta = True"
#   4. score = "aic-cg"    -> Solo para variables mixtas, es decir, cuando flag "discreta = False"
#   5. score = "bic-cg"    -> Solo para variables mixtas, es decir, cuando flag "discreta = False"
#   6. score = "loglik-cg" -> Solo para variables mixtas, es decir, cuando flag "discreta = False"
scoreR = "aic-cg"

# Este flag solo está diseñado para operar cuando el valor "flagModelo == 0"
# Permite indicar al aprendizaje de Estructura de "Python que algoritmo de puntuación utilizar
# Algoritmos de puntuación disponibles:
#   1. score = "bic"  -> Bayesian Information Criterion (Tambien llamado MDL)
#   2. score = "k2"   -> 
#   3. score = "bdeu" -> (DB) Bayesian Dirichlet, (e) for likelihood-equivalence, (u) for uniform joint distibution
scorePython = 'bdeu'

# Este flag permite usar la base de datos balanceada o no 
# balanceado == True  -> usar datos balanceados
# balanceado == False -> usar datos desbalanceados
balanceado = True

# Este flag permite usar la selección de variables
# seleccionVariables > 0  -> Hace uso de la selección de variables
# La configuración permitida son valores multiplos de 5 partiendo en 5 y terminando en 20
seleccionVariables = 0

# USO DE RESTRICCIONES
# =====================
# Para aplicar restricciones debe incluir las variables de a pares en las listas negras o blancas
# Ejemplos:
#  listaBlanca = ["tt", "game_score", "lt", "game_score", "pt", "game_score"]
#  listaNegra = ["estado", "programa", "game_score", "op1", "game_score", "sv1", "lt", "op5", "lt", "sv5", "lt", "op4", "lt", "sv4", "lt", "op2", "lt", "op3", "sv4", "op2", "op3", "sv2"]
#
# En el caso de la lista blanca está forzando a que existan los arcos: 
#  - ("tt", "game_score")
#  - ("lt", "game_score")
#  - ("pt", "game_score")
#
# Mismo caso ocuriría con la lista negra.
#
# SI NO SE QUIERE APLICAR LISTAS NEGRAS O BLANCAS ESTAS DEBEN ESTAR DECLARADAS VACIAS.
# COMO POR EJEMPLO:
listaBlanca = []
listaNegra = []

# BALANCEADO - MIXTO - AIC-CG
'''
listaBlanca = ["lt", "op1",
              "lt", "op2",
              "lt", "op3",
              "lt", "op4",
              "lt", "op5",
              "lt", "op6",
              "pt", "game_score"]
listaNegra = ["op1", "op5",
             "op1", "op4",
             "op1", "sv4",
             "op1", "op3",
             "op1", "op6",
             "op2", "sv3",
             "sv5", "op2",
             "op6", "op2",
             "sv6", "op2",
             "sv4", "op2",
             "sv5", "op3",
             "op6", "op3",
             "sv6", "op3",
             "sv4", "op3",
             "op5", "op3",
             "op4", "sv2",
             "sv6", "op4",
             "op5", "sv3",
             "op6", "sv2",
             "op6", "sv3"]
'''

# DESBALANCEADO - DISCRETO - AIC
# listaBlanca = ["op1", "op2", "programa", "estado"]
# listaNegra = ["lt", "sv4", "lt", "sv6", "op3", "sv2"]

# variable que identifica que variable del modelo será usada como clase a analizar.
varClase = 'estado'

### Validación de parámetros para el funcionamiento del modelo

In [2]:
if flagModelo == 1:
    print("Validando parámetros para ejecución de modelo en R")
    if (discreta == True) & ((scoreR == "aic") | (scoreR == "bic") | (scoreR == "loglik")):
        print("Parámetros de puntuación de aprendizaje de modelo 'R' OK")
    elif (discreta == False) & ((scoreR == "aic-cg") | (scoreR == "bic-cg") | (scoreR == "loglik-cg")):
        print("Parámetros de puntuación de aprendizaje de modelo 'R' OK")
    else:
        sys.exit("Parámetros de puntuación del modelo 'R' incorrectos, corrija el problema y vuelva a ejecutar")
else:
    print("Validando parámetros para ejecución de modelo en Python")
    if ((scorePython == "bic") | (scorePython == "k2") | (scorePython == "bdeu")):
        print("Parámetros de puntuación de aprendizaje de modelo 'Python' OK")
    else:
        sys.exit("Parámetros de puntuación del modelo 'Python' incorrectos, corrija el problema y vuelva a ejecutar")
        
if seleccionVariables in (0, 5, 10, 15, 20):
    print("Parámetros de selección de variables OK")
else:
    sys.exit("Parámetro de Selección de Variables debe ser 0, 5, 10, 15 o 20")   

Validando parámetros para ejecución de modelo en R
Parámetros de puntuación de aprendizaje de modelo 'R' OK
Parámetros de selección de variables OK


### Abriendo el conjunto de datos desde un archivo CSV y asignandolo al objeto DataFrame "df"

In [3]:
df = pd.read_csv('dataset_a.csv', sep=';', error_bad_lines=False)

## Transformación de variables cualitativas a continuas

### Etapas de la transformación

1. Se realiza la normalización de la variable "programa", que corresponde a una variable del tipo cualitativa/nominal.  También, se realiza la normalización de la variables "estado", que corresponde a una variable del tipo cualitativa/binaria.

2. Se realiza el reemplazo de los datos de las variables "programa" y "estado", utilizando la nornalización obtenida en la etapa anterior.  Con este reemplazo de datos las variables pasan a ser cuantitativas/continuas

3. Una vez que las variables mutaron, los datos quedan como tipo Object, por lo tanto se realiza un cast a int32 para ser usados.

In [4]:
#Etapa 1
normalizePrograma = df['programa'].value_counts(normalize=True, ascending=True)*100
if outInfo == True: 
    print('Pre-Normalización de variable "programa"')
    print('=======================================')
    print(normalizePrograma)
    print('\n')

normalizeEstado = df['estado'].value_counts(normalize=True, ascending=False)*100
if outInfo == True: 
    print('Pre-Normalización de variable "estado"')
    print('=====================================')
    print(normalizeEstado)
    print('\n')

#Etapa 2
for i in range(len(normalizePrograma)):
    df["programa"] = df["programa"].str.replace(normalizePrograma.index[i], str(i+1))

for i in range(len(normalizeEstado)):
    df["estado"] = df["estado"].str.replace(normalizeEstado.index[i], str(i))    

#Etapa 3
df['programa'] = df['programa'].astype('int')
df['estado'] = df['estado'].astype('int')

#Verificando que la transformación no haya afectado la normalización original
if outInfo == True:
    normalizePrograma = df['programa'].value_counts(normalize=True, ascending=True)*100
    print('Post-Normalización de variable "programa"')
    print('========================================')
    print(normalizePrograma)
    print('\n')

    normalizeEstado = df['estado'].value_counts(normalize=True, ascending=False)*100
    print('Post-Normalización de variable "estado"')
    print('======================================')
    print(normalizeEstado)

## Limpieza de variables

Se limpia el dataframe dejando solo las variables relevantes que, dado un estudio previo de la información, se determinó que eran relevantes y entregan un aporte real al modelo.

In [5]:
df = df.loc[:, ['lt', 'tt', 'pt', 'game_score', 
                'op1','op2', 'op3', 'op4', 'op5', 'op6',
                'sv1','sv2','sv3','sv4','sv5','sv6',
                'score', 'score_a', 'score_p', 'score_d', 'score_s',
                'programa', 'sol1', 'estado']]

if outInfo == True: print(df.columns)

## Discretizando variables cualitativas

Proporciona una forma de dividir características continuas en valores discretos.  La estrategia utilizada ("kmeans") 
permite elegir los "centroides" por cada "bins" establecido asignando cada muestra a su centroide más cercano, repitiendo el proceso hasta llegar a un umbral, es decir, se repite hasta que los centroides no se mueven significativamente.

En esta etapa se discretizaron las variables "lt", "tt", "pt", "game_score", "sol1".
Los "bins" fueron calculados a partir del metodo "bayesian_blocks" de la librería "astropy" que calcula la segmentación óptima de datos con los bloques bayesianos descritos por Scargle (2012).

K-means: https://scikit-learn.org/stable/modules/clustering.html#k-means

bloques bayesianos: https://ui.adsabs.harvard.edu/abs/2013ApJ...764..167S/abstract

In [6]:
if outInfo == True: 
    print('Pre-Discretización de variables')
    print('===============================')
    print(df.loc[:, ['lt', 'tt', 'pt', 'game_score', 'sol1']])
    print("\n")

# Obteniendo un "bins" optimo
bins = blU.bayesBlock(df["lt"])
if outInfo == True:    
    print("variable lt")
    print("===========")
    print("min : "+str(df["lt"].min()))
    print("max : "+str(df["lt"].max()))
    print("sum : "+str(df["lt"].sum()))
    print("bins: "+str(bins))
    print("\n")
valorDiscreto = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy = "kmeans").fit_transform(df[['lt']])
df[['lt']] = valorDiscreto

# Obteniendo un "bins" optimo
bins = blU.bayesBlock(df["tt"])
if outInfo == True: 
    print("variable tt")
    print("===========")
    print("min : "+str(df["tt"].min()))
    print("max : "+str(df["tt"].max()))
    print("sum : "+str(df["tt"].sum()))
    print("bins: "+str(bins))
    print("\n")
valorDiscreto = KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy = "kmeans").fit_transform(df[['tt']])
df[['tt']] = valorDiscreto

# Obteniendo un "bins" optimo
bins = blU.bayesBlock(df["pt"])
if outInfo == True: 
    print("variable pt")
    print("===========")
    print("min : "+str(df["pt"].min()))
    print("max : "+str(df["pt"].max()))
    print("sum : "+str(df["pt"].sum()))
    print("bins: "+str(bins))
    print("\n")
valorDiscreto =  KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy = "kmeans").fit_transform(df[['pt']])
df[['pt']] = valorDiscreto

# Obteniendo un "bins" optimo
bins = blU.bayesBlock(df["game_score"])
if outInfo == True: 
    print("variable game_score")
    print("===========")
    print("min : "+str(df["game_score"].min()))
    print("max : "+str(df["game_score"].max()))
    print("sum : "+str(df["game_score"].sum()))
    print("bins: "+str(bins))
    print("\n")
valorDiscreto =  KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy = "kmeans").fit_transform(df[['game_score']])
df[['game_score']] = valorDiscreto

# Obteniendo un "bins" optimo
bins = blU.bayesBlock(df["sol1"])
if outInfo == True: 
    print("variable sol1")
    print("===========")
    print("min : "+str(df["sol1"].min()))
    print("max : "+str(df["sol1"].max()))
    print("sum : "+str(df["sol1"].sum()))
    print("bins: "+str(bins))
    print("\n")
valorDiscreto =  KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy = "kmeans").fit_transform(df[['sol1']])
df[['sol1']] = valorDiscreto

if outInfo == True: 
    print('Post-Discretización de variables')
    print('===============================')
    print(df.loc[:, ['lt', 'tt', 'pt', 'game_score', 'sol1']])

## Ejecución de modelos de red bayesiana

Ejecución de modelos de red bayesiana que permiten obtener la causalidad Bayesiana por medio de la libreria "bnlearn", tanto de Python como de R

In [ ]:
if flagModelo == 0:
    modelos.modeloPython (df, varClase, 5, scorePython, balanceado, seleccionVariables)
    
if flagModelo == 1:   
    modelos.modeloR(df, varClase, 5, discreta, scoreR, balanceado, seleccionVariables, listaBlanca, listaNegra)

INICIO DE SECCION DE ENTRENAMIENTO, FOLD:  1
Balanceando porción de entrenamiento
